# Adaptive scaling with Dask Gateway

Motivation: Example compute showing adaptive scaling of workers, with dashboard plots.

In [ ]:
from dask_gateway import Gateway
# instantiate dask gateway
gateway = Gateway()

# view the cluster options UI
options = gateway.cluster_options()
options

By using the `Cluster Options` interface, we can specify the conda environment, the instance type, and any environment variables we might need. This is a visual example but all of this can of course be done programmatically:

```python
options.conda_environment = conda_env
options.profile = worker_type
options.environment_vars = {"MYENV": "aNeNvVaR"}
```

> ⚠️ Warning: It’s important that the environment used for your notebook (i.e. the kernel) must match the Dask worker environment (i.e. `options.conda_environment`).

Now to create a Dask cluster.

In [ ]:
# create a new cluster with our options
cluster = gateway.new_cluster(options)
# view the cluster UI
cluster

In [ ]:
# get the client for the cluster
client = cluster.get_client()
# view the client UI
# client

The `Dask Client` interface gives us a brief summary of everything we've set up so far. 

In [ ]:
# come up with better example to demo - use an actual dataset
import dask.array as da
x = da.random.random((100000, 100000), chunks=(1000, 1000))
y = x * x
z = y.mean(axis=1)
z.compute()

**ALWAYS** remember to shutdown your cluster!!

> ⚠️ Warning: As with JupyterLab servers, Dask workers run on cloud compute instances and cost actual money.

In [ ]:
cluster.close()
client.close()

---
## 👏 Next:
* [03_managing_environments](../03_managing_environments.ipynb)
---